### Use Antagonist to train a symptom detection model 

#### Dataset preparation

Note: the dataset needs to be downloaded using the script `download_SMD_dataset.sh` in the `scripts/antagonist-ml` folder.

In [1]:
from data_utils import SMD

In [2]:
db = SMD(dataset_folder=r"D:\antagonist\data\ServerMachineDataset")

In [6]:
dataframes = db.read_dataset(group_name="Group 1", train=False, retrieve_labels=False)
df = dataframes[0]

In [7]:
df

,cpu_r,load_1,load_5,load_15,mem_shmem,mem_u,mem_u_e,total_mem,disk_q,disk_r,...,out_rsts,out_segs,passive_opens,retransegs,tcp_timeouts,udp_in_dg,udp_out_dg,udp_rcv_buf_errs,udp_snd_buf_errs,timestamp
0,0.075269,0.065678,0.070234,0.074332,0.0,0.933333,0.274011,0.0,0.031081,0.000000,...,0.008596,0.068036,0.048893,0.000386,0.000034,0.064432,0.064500,0.0,0.0,2020-01-01 00:00:00
1,0.086022,0.080508,0.075808,0.076655,0.0,0.930769,0.274953,0.0,0.031081,0.000122,...,0.008596,0.070020,0.050437,0.000386,0.000022,0.065228,0.065224,0.0,0.0,2020-01-01 00:01:00
2,0.075269,0.064619,0.071349,0.074332,0.0,0.928205,0.274953,0.0,0.030940,0.000366,...,0.008596,0.069684,0.055069,0.000386,0.000045,0.067111,0.067178,0.0,0.0,2020-01-01 00:02:00
3,0.086022,0.048729,0.063545,0.070848,0.0,0.928205,0.273070,0.0,0.027250,0.000244,...,0.010029,0.073253,0.051467,0.000000,0.000034,0.066676,0.066744,0.0,0.0,2020-01-01 00:03:00
4,0.086022,0.051907,0.062430,0.070848,0.0,0.933333,0.274011,0.0,0.030940,0.000244,...,0.008596,0.070932,0.051467,0.000386,0.000022,0.066604,0.066671,0.0,0.0,2020-01-01 00:04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28474,0.075269,0.051907,0.047938,0.047619,0.0,0.907692,0.257062,0.0,0.043571,0.000244,...,0.031519,0.046733,0.040144,0.000000,0.000022,0.042931,0.043000,0.0,0.0,2020-01-20 18:34:00
28475,0.064516,0.025424,0.039019,0.044135,0.0,0.905128,0.257062,0.0,0.032501,0.000000,...,0.031519,0.047438,0.048893,0.000000,0.000056,0.046550,0.046619,0.0,0.0,2020-01-20 18:35:00
28476,0.064516,0.080508,0.050167,0.047619,0.0,0.907692,0.258004,0.0,0.026114,0.000611,...,0.031519,0.046797,0.040144,0.000386,0.000045,0.043003,0.043000,0.0,0.0,2020-01-20 18:36:00
28477,0.064516,0.056144,0.047938,0.046458,0.0,0.902564,0.257062,0.0,0.033210,0.000122,...,0.031519,0.041884,0.043232,0.000000,0.000045,0.039890,0.039959,0.0,0.0,2020-01-20 18:37:00


In [8]:
import plotly.graph_objects as go
from plotly_resampler import FigureWidgetResampler


fig = FigureWidgetResampler(go.Figure())

fig.add_trace(go.Scatter(x=df["timestamp"], y=df["load_1"], mode="lines", name="load_1"))

fig

FigureWidgetResampler({
    'data': [{'mode': 'lines',
              'name': '<b style="color:sandybrown">[R]</b> load_1 <i style="color:#fc9944">~28m</i>',
              'type': 'scatter',
              'uid': '6b170d1c-e917-4bd9-a50e-fc4f43bc1490',
              'x': array([datetime.datetime(2020, 1, 1, 0, 0),
                          datetime.datetime(2020, 1, 1, 0, 23),
                          datetime.datetime(2020, 1, 1, 0, 38), ...,
                          datetime.datetime(2020, 1, 20, 18, 2),
                          datetime.datetime(2020, 1, 20, 18, 25),
                          datetime.datetime(2020, 1, 20, 18, 38)], dtype=object),
              'y': array([0.065678, 0.034958, 0.190678, ..., 0.028602, 0.134534, 0.029661])}],
    'layout': {'template': '...'}
})